In [1]:
# !pip install onnx
# !pip install onnxruntime-gpu

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import PIL.Image as Image
import os
import glob
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# import matplotlib.pyplot as plt
import tqdm.notebook as notebook
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
# import onnx
# import onnxruntime as ort 
import pandas as pd

In [3]:
import os
base_path = '/kaggle/input/vesuvius-challenge-ink-detection/test'
test_fragments = sorted(os.listdir(base_path))
Z_START = 25
Z_DIM = 10
BUFFER = 30
BATCH_SIZE = 64
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data Processing

In [4]:
class SubvolumeDataset(Dataset):
    def __init__(self, image_stack, pixels):
        self.image_stack = image_stack
        self.pixels = pixels
    def __len__(self):
        return len(self.pixels)
    def __getitem__(self, index):
        y, x = self.pixels[index]
        subvolume = self.image_stack[:, y-BUFFER:y+BUFFER+1, x-BUFFER:x+BUFFER+1].view(1, Z_DIM, BUFFER*2+1, BUFFER*2+1)
        
        return subvolume

def pad_array(x,Buffer=BUFFER):
    x=  np.pad(x, ((0, 0), (Buffer, Buffer), (Buffer, Buffer)), mode='constant', constant_values=0)
    return x

In [5]:
class InkDetection(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv3d(in_channels = 1,out_channels = 16, kernel_size = 3 , padding = 1)
        self.act1 = nn.LeakyReLU()
        
        self.BN1 = nn.BatchNorm3d(16)
        self.conv2 =  nn.Conv3d(in_channels = 16,  out_channels =32 ,kernel_size = 3, padding =1)
        self.BN2 = nn.BatchNorm3d(32)
        self.conv3 =  nn.Conv3d(in_channels = 32,  out_channels =64 ,kernel_size = 3, padding =1)
        self.BN3 = nn.BatchNorm3d(64)
        
        self.pool = nn.MaxPool3d(2,2)
        
        self.ffn = nn.Sequential(nn.LazyLinear(128),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(128,128),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(128,2)
        )
        self.final = nn.Sigmoid()
        
    def forward(self,X):
        X= self.conv1(X)
        X= self.act1 (X)
        X=self.BN1(X)
        X=self.pool(X)
        X= self.conv2(X)
        X= self.act1 (X)
        X=self.BN2(X)
        X=self.pool(X)
        X= self.conv3(X)
        X= self.act1 (X)
        X=self.BN3(X)
        X=self.pool(X)

        X=X.flatten(start_dim=1)

        #linear layers
        X=self.ffn(X)
        X =self.final(X)


        return X

In [6]:
def model_evaluation(data, pixels):
    data = SubvolumeDataset(data,pixels)
    test_loader =  DataLoader(data, batch_size=BATCH_SIZE, shuffle=False)
    model = InkDetection().to(DEVICE)
    model = torch.load('/kaggle/input/d/harshapssh/model-batchnorm/model_batchnorm.pt',map_location=torch.device(DEVICE))
    model.load_state_dict(torch.load('/kaggle/input/d/harshapssh/model-batchnorm/model123_3_.pt',map_location=torch.device(DEVICE)))
    
    # Model Evaluation
#     torch.onnx.export(model.to(DEVICE),torch.randn(1,1,10,61,61,requires_grad=True).to(DEVICE),"model.onnx",export_params = True,input_names = ["x"],dynamic_axes = {"x": {0 :"Batch_size"}} ,opset_version = 10,do_constant_folding=True)
#     sess = ort.InferenceSession("/kaggle/working/model.onnx",providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'])
    
    progress_bar = notebook.tqdm(test_loader,desc = "Batch Number",leave =True)
    out_img = torch.zeros_like(mask).float()
    # with torch.no_grad():
    model.eval()
    for i,batch in enumerate(progress_bar):
        X =batch
#         output = sess.run(None,{"x" : X.cpu().numpy()})
        output = model(X.to(DEVICE))
#         output = torch.from_numpy(np.array(output)).squeeze(0)
        for j,value in enumerate(output):
            out_img[tuple(pixels[i*BATCH_SIZE+j])] = torch.argmax(value)
            
    out_img_a = out_img.cpu().numpy() #convert to Numpy array
    out_img_a = pd.DataFrame(out_img_a) #convert to a dataframe
    out_img_a.to_csv('test_'+frag+'_out_noisy.csv', index=False)

In [7]:
def rle (img):
    flat_img = img.flatten()
    flat_img = np.where(flat_img > 0.5, 1, 0).astype(np.uint8)

    starts = np.array((flat_img[:-1] == 0) & (flat_img[1:] == 1))
    ends = np.array((flat_img[:-1] == 1) & (flat_img[1:] == 0))
    starts_ix = np.where(starts)[0] + 2
    ends_ix = np.where(ends)[0] + 2
    lengths = ends_ix - starts_ix
    return starts_ix, lengths

In [8]:

for frag in test_fragments:
    test_path = base_path +'/' + frag
    
    mask = np.array(Image.open(test_path+"/mask.png"))
    PREFIX = test_path+"/surface_volume/"
    
    # Use glob.glob to get a list of all the file paths
    file_paths = [PREFIX + "{:02d}.tif".format(i) for i in range(Z_START, Z_START+Z_DIM)]

    # Load the images into a list of numpy arrays
    images = [np.array(Image.open(filename), dtype=np.float32)/65535.0 for filename in notebook.tqdm(file_paths)]
    data = np.array(images)
    data = pad_array(data)
    
    data = torch.stack([torch.from_numpy(image) for image in data], dim=0).to(DEVICE)
    mask = torch.from_numpy(np.pad(mask,((BUFFER, BUFFER), (BUFFER, BUFFER)), mode='constant', constant_values=0)).float().to(DEVICE)
    pixels = torch.argwhere(mask)
    
    model_evaluation(data, pixels)
    

  0%|          | 0/10 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Batch Number:   0%|          | 0/169046 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Batch Number:   0%|          | 0/286312 [00:00<?, ?it/s]

In [9]:
del data
del mask
del pixels
# del model

import gc
gc.collect()

153

In [10]:
output_str = 'Id,Predicted'

for frag in test_fragments:
    wk_dir = '/kaggle/working/'
    test = pd.read_csv('test_'+frag+'_out_noisy.csv').values
    test = test[30:-30, 30:-30]
    
    starts_ix, lengths = rle(test)
    inklabels_rle = " ".join(map(str, sum(zip(starts_ix, lengths), ())))
    output_str += "\n"+frag + "," + inklabels_rle

print(output_str, file=open('submission.csv', 'w'))